Copyright (c) 2020-2021. All rights reserved.

Licensed under the MIT License.

# HPO for Fine-Tuning Pre-trained Language Models


## 1. Introduction


In this notebook, we demonstrate a procedure for troubleshooting HPO failure in fine-tuning pre-trained language models (introduced in the following paper):

*An Empirical Study on Hyperparameter Optimization for Fine-Tuning Pre-trained Language Models. Xueqing Liu, Chi Wang. To appear in ACL-IJCNLP 2021*

FLAML requires `Python>=3.6`. To run this notebook example, please install flaml with the `notebook` and `nlp` options:
```bash
pip install flaml[notebook,nlp]
```

In [1]:
%cd /data/xliu127/projects/hyperopt/FLAML/
!pip install -e.[nlp]
!python setup.py install
import flaml
from flaml.nlp import AutoTransformers
import inspect
print(inspect.getsource(flaml.nlp))


/data/xliu127/projects/hyperopt/FLAML
Obtaining file:///data/xliu127/projects/hyperopt/FLAML


  Attempting uninstall: FLAML
    Found existing installation: FLAML 0.5.2
    Uninstalling FLAML-0.5.2:
      Successfully uninstalled FLAML-0.5.2
  Running setup.py develop for FLAML
running install
running bdist_egg
running egg_info
writing FLAML.egg-info/PKG-INFO
writing dependency_links to FLAML.egg-info/dependency_links.txt
writing requirements to FLAML.egg-info/requires.txt
writing top-level names to FLAML.egg-info/top_level.txt
reading manifest file 'FLAML.egg-info/SOURCES.txt'
adding license file 'LICENSE'
adding license file 'NOTICE.md'
writing manifest file 'FLAML.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying flaml/nlp/result_analysis/azure_utils.py -> build/lib/flaml/nlp/result_analysis
copying flaml/nlp/hpo/get_grid_search_space.py -> build/lib/flaml/nlp/hpo
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/flaml
creating build/bdist.linux-x86_64/egg/flaml/tune
copying bu

byte-compiling build/bdist.linux-x86_64/egg/flaml/tune/tune.py to tune.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/tune/sample.py to sample.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/tune/__init__.py to __init__.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/tune/analysis.py to analysis.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/tune/result.py to result.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/tune/trial_runner.py to trial_runner.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/config.py to config.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/ml.py to ml.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/version.py to version.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/searcher/blendsearch.py to blendsearch.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flaml/searcher/online_searcher.py to online_sea

from .hpo.hpo_searchspace import AutoHPOSearchSpace
from .autotransformers import AutoTransformers
from .result_analysis.azure_utils import AzureUtils, JobID



## 2. Initial Experimental Study (Section 4)


### Load dataset 

Load the dataset using AutoTransformer.prepare_data. In this notebook, we use the Recognizing Textual Entailment (RTE) dataset and the Electra model as an example:

In [2]:
from flaml.nlp.autotransformers import AutoTransformers
autohf = AutoTransformers()
preparedata_setting = {
        "dataset_subdataset_name": "glue:mrpc",
        "pretrained_model_size": "google/electra-base-discriminator:base",
        "data_root_path": "data/",
        "max_seq_length": 128,
        }
autohf.prepare_data(**preparedata_setting)


console_args has no attribute pretrained_model_size, continue
console_args has no attribute dataset_subdataset_name, continue
console_args has no attribute algo_mode, continue
console_args has no attribute space_mode, continue
console_args has no attribute search_alg_args_mode, continue
console_args has no attribute algo_name, continue
console_args has no attribute pruner, continue
console_args has no attribute resplit_mode, continue
console_args has no attribute rep_id, continue
console_args has no attribute seed_data, continue
console_args has no attribute seed_transformers, continue
console_args has no attribute varying_arg1, continue
console_args has no attribute varying_arg2, continue


Reusing dataset glue (/home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-1bfddd336b487fb5.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-c5a069956d4ac095.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-4fd1d046fef9604e.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-1bfddd336b487fb5.arrow
Loading cached processed dataset at /home/xliu127/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c996572

### Running grid search

First, we run grid search using Electra. By specifying `algo_mode="grid"`, AutoTransformers will run the grid search algorithm. By specifying `space_mode="grid"`, AutoTransformers will use the default grid search configuration recommended by the Electra paper:

In [3]:
autohf_settings = {"resources_per_trial": {"gpu": 1, "cpu": 1},
                   "num_samples": 1,
                   "time_budget": 100000,  # unlimited time budget
                   "ckpt_per_epoch": 5,
                   "fp16": True,
                   "algo_mode": "grid",  # set the search algorithm to grid search
                   "space_mode": "grid", # set the search space to the recommended grid space
                   }
validation_metric, analysis = autohf.fit(**autohf_settings,)

2021-06-13 20:18:18,528	INFO tune.py:450 -- Total run time: 141.76 seconds (141.60 seconds for the tuning loop).


Get the time for running grid search: 

In [4]:
GST = autohf.last_run_duration
print("grid search for {} took {} seconds".format(autohf.jobid_config.get_jobid_full_data_name(), GST))

grid search for glue_mrpc took 141.79089450836182 seconds


After the HPO run finishes, generate the predictions and save it as a .zip file to be submitted to the glue website. Here we will need the library AzureUtils which is for storing the output information (e.g., analysis log, .zip file) locally and uploading the output to an azure blob container (e.g., if multiple jobs are executed in a cluster). If the azure key and container information is not specified, the output information will only be saved locally. 

In [5]:
predictions, test_metric = autohf.predict()
from flaml.nlp.result_analysis.azure_utils import AzureUtils

print(autohf.jobid_config)

azure_utils = AzureUtils(root_log_path="logs_test/", autohf=autohf)
azure_utils.write_autohf_output(valid_metric=validation_metric,
                                predictions=predictions,
                                duration= autohf.last_run_duration)
print(validation_metric)

remove_columns_ is deprecated and will be removed in the next major version of datasets. Use the dataset.remove_columns method instead.


JobID(dat=['glue'], subdat='mrpc', mod='grid', spa='grid', arg='dft', alg='grid', pru='None', pre_full='google/electra-base-discriminator', pre='electra', presz='base', spt='ori', rep=0, sddt=43, sdhf=42, var1=None, var2=None)
_container_name is unspecified or wrongly specified, please specify _container_name in AzureUtils
console_args does not contain data_root_dir, loading the default value
_container_name is unspecified or wrongly specified, please specify _container_name in AzureUtils


The validation F1/accuracy we got was 92.8/90.0. After the above steps, you will find a .zip file for the predictions under data/result/. Submit the .zip file to the glue website. The test F1/accuracy we got was 90.9/87.8.

### Running Random Search

Next, we run random search with the same time budget as grid search:

In [1]:
hpo_space = {
               "learning_rate": {"l": 3e-5, "u": 1.5e-5, "space": "log"},
               "warmup_ratio": {"l": 0, "u": 0.2, "space": "linear"},
               "num_train_epochs": [3],
               "per_device_train_batch_size": [16, 32, 64],
               "weight_decay": {"l": 0.0, "u": 0.3, "space": "linear"},
               "attention_probs_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"},
               "hidden_dropout_prob": {"l": 0, "u": 0.2, "space": "linear"}
            }

def tune_hpo(time_budget):
    autohf_settings = {"resources_per_trial": {"gpu": 1, "cpu": 1},
                       "num_samples": -1,
                       "time_budget": time_budget,  # unlimited time budget
                       "ckpt_per_epoch": 5,
                       "fp16": True,
                       "algo_mode": "hpo",  # set the search algorithm to grid search
                       "algo_name": "rs",
                       "space_mode": "cus", # set the search space to the recommended grid space
                       "hpo_space": hpo_space
                       }
    validation_metric, analysis = autohf.fit(**autohf_settings,)
    predictions, test_metric = autohf.predict()
    azure_utils = AzureUtils(root_log_path="logs_test/", autohf=autohf)
    azure_utils.write_autohf_output(valid_metric=validation_metric,
                                    predictions=predictions,
                                    duration= autohf.last_run_duration)
    print(validation_metric)

tune_hpo(GST)

NameError: name 'GST' is not defined

The validation F1/accuracy we got was 92.6/89.7. Similarly, we can submit the .zip file to the glue website. The test F1/accuaracy we got was 90.4/86.8. Since the validation accuracy is smaller than grid search, we continue increasing the time budget for HPO.

## 3. Troubleshooting HPO Failures

Since the validation accuracy is smaller than grid search, we increase the time budget for HPO:

In [ ]:
GST = 600
tune_hpo(4 * GST)